# Connect 4

In [ ]:
from collections import Counter
import numpy as np

In [ ]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

## Basic Functions

In [ ]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

## Montecarlo Evaluation

In [ ]:
#importante
def _mcSquillero(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0

def montecarloSquillero(board, player, plays):
    montecarlo_samples = plays
    cnt = Counter(_mcSquillero(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples

def tryMove(board, player):
    mov = montecarloSquillero(board, player, 25)*player
    #print(mov, " ",player,'\t')
    counterBest = -20
    for c in valid_moves(board):
        fakeBoard = np.copy(board)
        play(fakeBoard, c, -player)
        tmp = montecarloSquillero(fakeBoard, -player, 20)*(-player)
        if tmp == 1.0:
            tmp == 10
        if counterBest < tmp :
            counterBest = tmp
    print(mov, " ",player,'\t', counterBest, " ",-player, "  ->   ", mov - counterBest, '\n')
    return (mov - counterBest)

def _mc(board, player):

    best = -20
    col = 0
    print("Starting best: ", best)
    for c in valid_moves(board):
        fakeBoard = np.copy(board)
        play(fakeBoard, c, player)
        tmp = tryMove(fakeBoard, player)
        if best < tmp :
            best = tmp
            col = c
        print("Actual best: ", best,' ', col)
            
    play(board, col, player)
    if four_in_a_row(board, player):
        return player
    return 0    
    
def eval_board(board, player):
    if four_in_a_row(board, 1):
    # Alice won
        return 1
    elif four_in_a_row(board, -1):
    # Bob won
        return -1
    else:
    # Not terminal, let's simulate...
        return _mc(board, player)

## Example

In [ ]:
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)

print(board)
print('\n')
p=-1
tmp = 0
while tmp!=1 and tmp!=-1:
    p=-p
    tmp = eval_board(board, p)
    print(board)
    print('\n')
